In [1]:
import scipy
import numpy as np
import pandas as pd
from scipy import special
from scipy.optimize import minimize
from scipy.interpolate import interp1d

from mpmath import invertlaplace
from mpmath import *
mp.dps = 10; mp.pretty = True

from scipy.stats import pearsonr
from sklearn.metrics import r2_score

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def simulate_HSDM_4D(a, mu, ndt, sigma=1, dt=0.001):
    x = np.zeros(mu.shape)
    
    rt = 0
    
    while np.linalg.norm(x, 2) < a(rt):
        x += mu*dt + sigma*np.sqrt(dt)*np.random.normal(0, 1, mu.shape)
        rt += dt
    
    theta1 = np.arctan2(np.sqrt(x[3]**2 + x[2]**2 + x[1]**2), x[0])
    theta2 = np.arctan2(np.sqrt(x[3]**2 + x[2]**2), x[1])
    theta3 = np.arctan2(x[3], x[2])
    
    
    return rt+ndt, (theta1, theta2, theta3)

In [3]:
def k(a, da, t, q, sigma=2):
    return 0.5 * (q - 0.5*sigma - da(t))

def psi(a, da, t, z, tau, q, sigma=2):
    kk = k(a, da, t, q, sigma)
    
    term1 = 1./(sigma*(t - tau)) * np.exp(- (a(t) + z)/(sigma*(t-tau)))
    term2 = (a(t)/z)**(0.5*(q-sigma)/sigma)
    term3 = da(t) - (a(t)/(t-tau)) + kk
    term4 = special.iv(q/sigma-1, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
    term5 = (np.sqrt(a(t)*z)/(t-tau)) * special.iv(q/sigma, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
    
    return term1 * term2 * (term3 * term4 + term5)

def ie_bessel_fpt(a, da, q, z, dt=0.1, T_max=2):
    g = [0]
    T = [0]
    g.append(-2*psi(a, da, dt, z, 0, q))
    T.append(dt)
    
    for n in range(2, int(T_max/dt)+2):
        s = -2 * psi(a, da, n*dt, z, 0, q)

        for j in range(1, n):
            s += 2 * dt * g[j] * psi(a, da, n*dt, a(j*dt), j*dt, q)

        g.append(s)
        T.append(n*dt)
        
    g = np.asarray(g)
    T = np.asarray(T)
    
    gt = interp1d(T, g)
    return gt

In [4]:
def HSDM_4D_likelihood(prms, RT, Theta):
    ndt = prms[1]
    mu = np.array([prms[2], prms[3], prms[4], prms[5]])
    
    if prms[0] <= 2:
        a = lambda t: prms[0]**2
        da = lambda t: 0
        fpt = ie_bessel_fpt(a, da, mu.shape[0], 0.001, 
                            dt=0.01, T_max=max(RT))
    else:
        a = prms[0]
        nu = (mu.shape[0]-2)/2
        fixed = 2**nu * gamma(1+nu)
        fpt = lambda p: (a*sqrt(2*p))**nu/fixed *  1./besseli(nu, a*sqrt(2*p))
    
    
    log_lik = 0
    for i in range(len(RT)):
        rt, theta = RT[i], Theta[i]
        if rt - ndt > 0.001:
            mu_dot_x0 = mu[0]*np.cos(theta[0])
            mu_dot_x1 = mu[1]*np.sin(theta[0])*np.cos(theta[1]) 
            mu_dot_x2 = mu[2]*np.sin(theta[0])*np.sin(theta[1])*np.cos(theta[2])
            mu_dot_x3 = mu[3]*np.sin(theta[0])*np.sin(theta[1])*np.sin(theta[2]) 
            term1 = prms[0] * (mu_dot_x0 + mu_dot_x1 + mu_dot_x2 + mu_dot_x3)
            term2 = 0.5 * np.linalg.norm(mu, 2)**2 * (rt-ndt)
            
            if prms[0] < 2: 
                density = exp(term1 - term2) * fpt(rt - ndt)

                if 0.1**14 < density:
                    log_lik += -log(density)
                else:
                    log_lik += -log(0.1**14)
            else:
                density = exp(term1 - term2) * invertlaplace(fpt, rt-ndt, method='talbot')
                if 0.1**14 < density:
                    log_lik += -log(density)
                else:
                    log_lik += -log(0.1**14)
        else:
            log_lik += -np.log(0.1**14)
        
    return log_lik

In [5]:
recovery_df = {'threshold_true': [],
               'threshold_estimate': [],
               'ndt_true': [],
               'ndt_estimate': [],
               'mu1_true': [],
               'mu1_estimate': [],
               'mu2_true': [],
               'mu2_estimate': [],
               'mu3_true': [],
               'mu3_estimate': [],
               'mu4_true': [],
               'mu4_estimate': []}

min_threshold = 0.5
max_threshold = 5

min_ndt = 0.1
max_ndt = 1

min_mu = -3.5
max_mu = 3.5

In [6]:
for n in range(3):
    threshold = np.random.uniform(min_threshold, max_threshold)
    a = lambda t: threshold
    ndt = np.random.uniform(min_ndt, max_ndt)
    mu = np.array([np.random.uniform(min_mu, max_mu), 
                   np.random.uniform(min_mu, max_mu),
                   np.random.uniform(min_mu, max_mu),
                   np.random.uniform(min_mu, max_mu)])
    
    
    recovery_df['threshold_true'].append(threshold)
    recovery_df['ndt_true'].append(ndt)
    recovery_df['mu1_true'].append(mu[0])
    recovery_df['mu2_true'].append(mu[1])
    recovery_df['mu3_true'].append(mu[2])
    recovery_df['mu4_true'].append(mu[3])
    
    RT = []
    Theta = []
    
    print(n+1, end=': ')
    for i in range(250):
        rt, theta = simulate_HSDM_4D(a, mu, ndt)
        RT.append(rt)
        Theta.append(theta)
    
    print('Simulation is done!', end=' / ')
    
    min_ans = minimize(HSDM_4D_likelihood,
                       args=(RT, Theta), 
                       x0=np.array([np.random.uniform(min_threshold, max_threshold),
                                    np.random.uniform(min_ndt, max_ndt), 
                                    np.random.uniform(min_mu, max_mu),
                                    np.random.uniform(min_mu, max_mu),
                                    np.random.uniform(min_mu, max_mu),
                                    np.random.uniform(min_mu, max_mu)]),
                       method='Nelder-Mead', 
                       bounds=[(min_threshold, max_threshold), (min_ndt, max_ndt),
                               (min_mu, max_mu), (min_mu, max_mu), 
                               (min_mu, max_mu), (min_mu, max_mu)])
    
    recovery_df['threshold_estimate'].append(min_ans.x[0])
    recovery_df['ndt_estimate'].append(min_ans.x[1])
    recovery_df['mu1_estimate'].append(min_ans.x[2])
    recovery_df['mu2_estimate'].append(min_ans.x[3])
    recovery_df['mu3_estimate'].append(min_ans.x[4])
    recovery_df['mu4_estimate'].append(min_ans.x[5])
    
    print('Estimation is done!')
    
recovery_df = pd.DataFrame(recovery_df)

1: Simulation is done! / Estimation is done!
2: Simulation is done! / Estimation is done!
3: Simulation is done! / Estimation is done!


In [7]:
recovery_df

,threshold_true,threshold_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate,mu3_true,mu3_estimate,mu4_true,mu4_estimate
0,1.449251,2.192229,0.241117,0.100000,2.419509,3.500000,-1.767270,-1.698834,2.250666,2.356766,-2.027601,-1.967094
1,1.561064,1.599150,0.411789,0.432109,-3.439085,-3.500000,-1.630062,-3.500000,0.694053,0.671501,-1.406372,-1.584185
2,3.541350,3.425129,0.285463,0.284340,-2.876523,-2.742637,-1.313955,-1.162310,-2.400211,-2.279225,-2.104045,-1.994028


In [8]:
recovery_df.corr()

,threshold_true,threshold_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate,mu3_true,mu3_estimate,mu4_true,mu4_estimate
threshold_true,1.000000,0.931780,-0.221159,0.110783,-0.465312,-0.455009,0.968402,0.641877,-0.958965,-0.949308,-0.544787,-0.509719
threshold_estimate,0.931780,1.000000,-0.560105,-0.257562,-0.112241,-0.100702,0.811803,0.876457,-0.790619,-0.770432,-0.812043,-0.787269
ndt_true,-0.221159,-0.560105,1.000000,0.944734,-0.760320,-0.767807,0.029047,-0.889777,-0.064420,-0.096615,0.938294,0.951766
ndt_estimate,0.110783,-0.257562,0.944734,1.000000,-0.931247,-0.935413,0.355141,-0.690978,-0.388016,-0.417579,0.773060,0.798577
mu1_true,-0.465312,-0.112241,-0.760320,-0.931247,1.000000,0.999933,-0.671359,0.380064,0.697179,0.719968,-0.488766,-0.524349
mu1_estimate,-0.455009,-0.100702,-0.767807,-0.935413,0.999933,1.000000,-0.662713,0.390772,0.688812,0.711866,-0.498857,-0.534195
mu2_true,0.968402,0.811803,0.029047,0.355141,-0.671359,-0.662713,1.000000,0.430359,-0.999373,-0.997708,-0.318438,-0.279050
mu2_estimate,0.641877,0.876457,-0.889777,-0.690978,0.380064,0.390772,0.430359,1.000000,-0.398129,-0.368296,-0.992711,-0.986893
mu3_true,-0.958965,-0.790619,-0.064420,-0.388016,0.697179,0.688812,-0.999373,-0.398129,1.000000,0.999478,0.284675,0.244876
mu3_estimate,-0.949308,-0.770432,-0.096615,-0.417579,0.719968,0.711866,-0.997708,-0.368296,0.999478,1.000000,0.253567,0.213435


In [9]:
recovery_df.to_csv('Hybrid_4d_recovery.csv')